在第一版本的思路中，我们主要的工作如下：

+ 使用更加精简的数据结构来存储稀疏矩阵，不调用任何其他的库进行处理，然后我们不存储0值
+ 为了适配稀疏矩阵的数据结构，我们使用了适配的算法，每次进入处理的相关内存量较小。

第二版本，**我们使用分批的思想**，就是程序运行的时候我们每个时间内调用的内存（就是稀疏矩阵进入RAM的值被限定），通过分批次的思想对算法处理进行进一步的优化！

## 读取部分

分批的话，我们一次就不能够直接读取所有数据，我们限定我们读取的数据量。

In [3]:
def read_lines_data(file_path):
    line_number = 0
    with open(file_path, 'r') as file:
        for line in file:
            line_number += 1
    print("Total lines: ", line_number)

file_path = "Data.txt"
read_lines_data(file_path)

Total lines:  150000


In [2]:
transM = {}

def read_data(file_path):
    with open(file_path, "r") as file:
        for line in file:
            parts = line.strip().split()
            from_node_id = int(parts[0])
            to_node_id = int(parts[1])
            if from_node_id not in transM:
                transM[from_node_id] = {"degree": 1, "dest": [to_node_id]}
            else:
                transM[from_node_id]["degree"] += 1
                transM[from_node_id]["dest"].append(to_node_id)
            if to_node_id not in transM:
                transM[to_node_id] = {"degree": 0, "dest": []}


file_path = "Data.txt"
read_data(file_path)
i = 10000

for item in transM:
    i = i - 1
    if i == 0:
        break
    print(item, transM[item])

6539 {'degree': 25, 'dest': [5619, 8970, 6279, 7182, 4873, 9630, 68, 6542, 744, 3387, 9592, 9438, 3087, 5706, 1990, 8146, 4430, 7041, 4805, 7643, 9725, 6375, 9091, 4326, 2716]}
5619 {'degree': 11, 'dest': [7019, 295, 3044, 1979, 3142, 4891, 4672, 3119, 2030, 6626, 7692]}
1840 {'degree': 21, 'dest': [6885, 7121, 2429, 5723, 1503, 3731, 869, 5253, 6302, 4399, 1058, 4360, 7377, 9793, 1263, 5175, 3703, 4590, 5826, 8128, 3290]}
6885 {'degree': 17, 'dest': [9883, 6964, 9714, 7567, 7186, 7918, 7845, 3569, 5749, 2887, 1887, 9629, 3636, 2140, 2547, 360, 9712]}
1303 {'degree': 22, 'dest': [8932, 4119, 7583, 5876, 9641, 233, 5096, 6487, 689, 6357, 990, 5666, 7667, 4121, 6920, 4237, 8506, 4190, 8573, 1434, 4134, 1654]}
8932 {'degree': 10, 'dest': [948, 5958, 4682, 2739, 6530, 1066, 5620, 7622, 3719, 5020]}
2203 {'degree': 21, 'dest': [4493, 3103, 4746, 6600, 322, 119, 1828, 2585, 6456, 9099, 8465, 5904, 4749, 5782, 7246, 4140, 7475, 7744, 3289, 6607, 3468]}
4493 {'degree': 20, 'dest': [7302, 8587,

In [4]:
import numpy as np # 我们看看如果不是用numpy库是否会降低内存占用


def converge(r_old, r_new, epsilon):
    return np.sum(np.abs(r_new - r_old)) >= epsilon  # 返回True表示继续迭代


HashDict = {} # 建立哈希表，我们没有必要存储ReverseDict，这个也是占用内存的,于是我们相对于第一版本减少了1点点，好像影响不大
# 假设transM已定义且包含所有节点信息
ii = 0
for key in transM:
    HashDict[key] = ii
    ii += 1
length = len(HashDict)  # 确保length正确初始化


def HashMap(Matrixkey):
    return HashDict[Matrixkey]


def ReverseHashMap(index):
    for key,value in HashDict.items():
        if value == index:
            return key


def pagerank(Matrix, epsilon=0.0001, beta=0.85):
    r_old = np.array([1.0 / length] * length)
    iteration_count = 0  # 可选：跟踪迭代次数

    while True:
        r_new = np.array(
            [(1 - beta) / length] * length, dtype=np.float64
        )  # 每次迭代重置基值

        for i in range(length):
            source = ReverseHashMap(i)
            degree = Matrix[source]["degree"]

            if degree == 0:
                # 处理悬挂节点：贡献均分给所有节点
                contribution = beta * r_old[i] / length
                r_new += contribution
            else:
                contribution = beta * r_old[i] / degree
                for dest in Matrix[source]["dest"]:
                    j = HashMap(dest)
                    r_new[j] += contribution

        if not converge(r_old, r_new, epsilon):
            break

        r_old = r_new.copy()
        iteration_count += 1

    return r_new


def get_top_100_indices(r_new):
    indexed_values = [(index, value) for index, value in enumerate(r_new)]
    indexed_values.sort(key=lambda x: x[1], reverse=True)
    top_100_indices = [
        (ReverseHashMap(index), value) for index, value in indexed_values[:100]
    ]
    return top_100_indices


top_100 = get_top_100_indices(pagerank(transM))
print(top_100)

[(286, np.float64(0.00019800862511586278)), (3473, np.float64(0.0001957968674538886)), (4951, np.float64(0.00019249583408793288)), (3890, np.float64(0.00019106286470993428)), (7365, np.float64(0.00018895632926081962)), (6359, np.float64(0.00018761017967043662)), (4352, np.float64(0.00018199874689874956)), (7032, np.float64(0.00018147135913026985)), (7541, np.float64(0.00018102468426217037)), (3699, np.float64(0.00018046757763955955)), (4877, np.float64(0.00017851557668676176)), (3242, np.float64(0.00017668630309151348)), (6503, np.float64(0.00017576274337224626)), (4221, np.float64(0.00017494971888432182)), (7293, np.float64(0.00017425519865568726)), (2276, np.float64(0.00017397898369288497)), (1189, np.float64(0.00017396523198836324)), (1441, np.float64(0.00017157359794528227)), (1866, np.float64(0.00017076440802799535)), (8602, np.float64(0.0001702992018069148)), (5012, np.float64(0.00017029846926393775)), (2342, np.float64(0.000170165524816598)), (4660, np.float64(0.0001695612752817

In [5]:
import numpy as np


def converge(r_old, r_new, epsilon):
    return np.sum(np.abs(r_new - r_old)) >= epsilon  # 返回True表示继续迭代


HashDict = {}
ReverseDict = {}
# 假设transM已定义且包含所有节点信息
ii = 0
for key in transM:
    HashDict[key] = ii
    ReverseDict[ii] = key
    ii += 1
length = len(HashDict)  # 确保length正确初始化


def HashMap(Matrixkey):
    return HashDict[Matrixkey]


def ReverseHashMap(index):
    return ReverseDict[index]


def pagerank(Matrix, epsilon=0.0001, beta=0.85):
    r_old = np.array([1.0 / length] * length)
    iteration_count = 0  # 可选：跟踪迭代次数

    while True:
        r_new = np.array(
            [(1 - beta) / length] * length, dtype=np.float64
        )  # 每次迭代重置基值

        for i in range(length):
            source = ReverseHashMap(i)
            degree = Matrix[source]["degree"]

            if degree == 0:
                # 处理悬挂节点：贡献均分给所有节点
                contribution = beta * r_old[i] / length
                r_new += contribution
            else:
                contribution = beta * r_old[i] / degree
                for dest in Matrix[source]["dest"]:
                    j = HashMap(dest)
                    r_new[j] += contribution

        if not converge(r_old, r_new, epsilon):
            break

        r_old = r_new.copy()
        iteration_count += 1

    return r_new


def get_top_100_indices(r_new):
    indexed_values = [(index, value) for index, value in enumerate(r_new)]
    indexed_values.sort(key=lambda x: x[1], reverse=True)
    top_100_indices = [
        (ReverseHashMap(index), value) for index, value in indexed_values[:100]
    ]
    return top_100_indices


top_100 = get_top_100_indices(pagerank(transM))
print(top_100)

[(286, np.float64(0.00019800862511586278)), (3473, np.float64(0.0001957968674538886)), (4951, np.float64(0.00019249583408793288)), (3890, np.float64(0.00019106286470993428)), (7365, np.float64(0.00018895632926081962)), (6359, np.float64(0.00018761017967043662)), (4352, np.float64(0.00018199874689874956)), (7032, np.float64(0.00018147135913026985)), (7541, np.float64(0.00018102468426217037)), (3699, np.float64(0.00018046757763955955)), (4877, np.float64(0.00017851557668676176)), (3242, np.float64(0.00017668630309151348)), (6503, np.float64(0.00017576274337224626)), (4221, np.float64(0.00017494971888432182)), (7293, np.float64(0.00017425519865568726)), (2276, np.float64(0.00017397898369288497)), (1189, np.float64(0.00017396523198836324)), (1441, np.float64(0.00017157359794528227)), (1866, np.float64(0.00017076440802799535)), (8602, np.float64(0.0001702992018069148)), (5012, np.float64(0.00017029846926393775)), (2342, np.float64(0.000170165524816598)), (4660, np.float64(0.0001695612752817